In [1]:
import sympy as sym

import repeated_play

In [2]:
pcc, pcd, pdc, pdd = sym.symbols("p_{CC}, p_{CD}, p_{DC}, p_{DD}")

In [3]:
qcc, qcd, qdc, qdd = sym.symbols("q_{CC}, q_{CD}, q_{DC}, q_{DD}")

In [6]:
R, S, T, P = sym.symbols("R, S, T, P")

In [32]:
M = repeated_play.transition_matrix_repeated_game([pcc, pcd, pdc, pdd],
                                                  [qcc, qcd, qdc, qdd],
                                                 memory="one", analytical=True)

In [25]:
ss = repeated_play.stationary_distribution(M, analytical=True)

In [26]:
expr = sum(ss @ sym.Matrix([R, S, T, P]))

In [27]:
expr = expr.factor()

In [52]:
new_M = M.copy() - sym.eye(4, 4)

In [57]:
new_M[:, 0] + new_M[:, 1] 

Matrix([
[p_{CC}*q_{CC} + p_{CC}*(1 - q_{CC}) - 1],
[p_{CD}*q_{DC} + p_{CD}*(1 - q_{DC}) - 1],
[    p_{DC}*q_{CD} + p_{DC}*(1 - q_{CD})],
[    p_{DD}*q_{DD} + p_{DD}*(1 - q_{DD})]])

In [58]:
for i in range(4):
    new_M[i, 1] = (new_M[i, 1] + new_M[i, 0]).factor()
    new_M[i, 2] = (new_M[i, 2] + new_M[i, 0]).factor()

In [61]:
new_M[:, 3] = sym.Matrix([R, S, T, P])

In [66]:
new_M2 = new_M.copy()

new_M2[:, 3] = sym.Matrix([1, 1, 1, 1])

In [67]:
expr2 = new_M.det() / new_M2.det()

In [68]:
diff = expr - expr2

In [69]:
diff.factor()

0

### With errors

In [199]:
epsilon = sym.symbols("\epsilon")

In [200]:
strategy = (sym.ones(4, 1) - sym.Matrix([1, pcd, pdc, pdd])) * epsilon + sym.Matrix([1, pcd, pdc, pdd]) * (1 - epsilon)

In [201]:
strategy = list(strategy.T)

In [202]:
coplayer = (sym.ones(4, 1) - sym.Matrix([1, qcd, qdc, qdd])) * epsilon + sym.Matrix([1, qcd, qdc, qdd]) * (1 - epsilon)

In [203]:
coplayer = list(coplayer.T)

In [204]:
M = repeated_play.transition_matrix_repeated_game(strategy, coplayer,
                                                 memory="one", analytical=True)

In [205]:
ss = repeated_play.stationary_distribution(M, analytical=True)

In [206]:
expr = sum(ss @ sym.Matrix([R, S, T, P]))

In [207]:
expr = expr.factor()

In [208]:
new_M = M.copy() - sym.eye(4, 4)

In [209]:
for i in range(4):
    new_M[i, 1] = (new_M[i, 1] + new_M[i, 0]).factor()
    new_M[i, 2] = (new_M[i, 2] + new_M[i, 0]).factor()

In [210]:
new_M[:, 3] = sym.Matrix([R, S, T, P])

In [211]:
new_M2 = new_M.copy()

new_M2[:, 3] = sym.Matrix([1, 1, 1, 1])

In [212]:
new_M

Matrix([
[                                                                          (1 - \epsilon)**2 - 1,                                  -\epsilon,                                  -\epsilon, R],
[(\epsilon*(1 - p_{CD}) + p_{CD}*(1 - \epsilon))*(\epsilon*(1 - q_{DC}) + q_{DC}*(1 - \epsilon)), -2*\epsilon*p_{CD} + \epsilon + p_{CD} - 1,     -2*\epsilon*q_{DC} + \epsilon + q_{DC}, S],
[(\epsilon*(1 - p_{DC}) + p_{DC}*(1 - \epsilon))*(\epsilon*(1 - q_{CD}) + q_{CD}*(1 - \epsilon)),     -2*\epsilon*p_{DC} + \epsilon + p_{DC}, -2*\epsilon*q_{CD} + \epsilon + q_{CD} - 1, T],
[(\epsilon*(1 - p_{DD}) + p_{DD}*(1 - \epsilon))*(\epsilon*(1 - q_{DD}) + q_{DD}*(1 - \epsilon)),     -2*\epsilon*p_{DD} + \epsilon + p_{DD},     -2*\epsilon*q_{DD} + \epsilon + q_{DD}, P]])

In [111]:
det1 = new_M.det()

In [112]:
det2 = new_M2.det()

In [113]:
expr2 = det1 / det2

In [114]:
diff = expr - expr2

In [115]:
diff.factor()

0

### Reactive 2

In [217]:
from sympy.polys.domainmatrix import DomainMatrix

In [218]:
pcc, pcd, pdc, pdd = sym.symbols("p_{CC}, p_{CD}, p_{DC}, p_{DD}")

In [316]:
e = sym.symbols("e")

In [219]:
b, c = sym.symbols("b, c")

In [220]:
strategy = [1 * (1 - epsilon) + 0 * epsilon,
            pcd * (1 - epsilon) + (1 - pcd) * epsilon,
            1 * (1 - epsilon) + 0 * epsilon,
            pcd * (1 - epsilon) + (1 - pcd) * epsilon,
            pdc * (1 - epsilon) + (1 - pdc) * epsilon,
            pdd * (1 - epsilon) + (1 - pdd) * epsilon,
            pdc * (1 - epsilon) + (1 - pdc) * epsilon,
            pdd * (1 - epsilon) + (1 - pdd) * epsilon]

In [221]:
strategy = strategy * 2

In [234]:
M = repeated_play.transition_matrix_repeated_game(strategy,
                                                  strategy,
                                                  memory='two',
                                                  analytical=True)

In [254]:
new_M = M.copy() - sym.eye(16, 16)

In [257]:
new_M[:, -1] = sym.Matrix([0 for _ in range(16)])

In [323]:
expr.subs({pcd:0, pdc:1, pdd:0, e:0.0001})

0.500000000000119

## Numeric stuff to check mathematica

In [259]:
import numpy as np

In [301]:
player = np.array([1, 0, 1, 0]) # np.random.random(4)
player[0] = 1

In [302]:
player

array([1, 0, 1, 0])

In [303]:
epsilon_val = 0.001

In [304]:
player = list(player * (1 - epsilon_val) + (1 - player) * epsilon_val)

In [305]:
player = [player[0], player[1], player[0], player[1], player[2], player[3], player[2], player[3]] * 2

In [306]:
M = repeated_play.transition_matrix_repeated_game(player, player, memory="two")

In [307]:
ss = repeated_play.stationary_distribution(M)[0]

In [308]:
ss[0] + ss[1] + ss[4] + ss[5] + ss[8] + ss[9] + ss[12] + ss[13]

0.49999999999998346

In [309]:
ss[0]

0.24950024999996803